In [4]:
import requests
import lxml.html as lh
import pandas as pd
from bs4 import BeautifulSoup
import urllib
import html2text
import nltk   

In [20]:
from bs4 import BeautifulSoup
import urllib

def urltorefseq(url):
    
    fullurl = "https://www.ncbi.nlm.nih.gov/genome/?term=" + url
    
    html = urllib.request.urlopen(fullurl).read()
    soup = BeautifulSoup(html)

    # kill all script and style elements
    for script in soup(["script", "style"]):
        script.extract()    # rip it out

    # get text
    maintext = soup.get_text()

    # break into lines and remove leading and trailing space on each
    lines = (line.strip() for line in maintext.splitlines())
    # break multi-headlines into a line each
    chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
    # drop blank lines
    maintext = '\n'.join(chunk for chunk in chunks if chunk)
    
    idpos = maintext.find("ID:")
    
    if(idpos == -1):
        
        newurl = "https://www.ncbi.nlm.nih.gov/bioproject/" + url
        
        html = urllib.request.urlopen(newurl).read()
        soup = BeautifulSoup(html)

        # kill all script and style elements
        for script in soup(["script", "style"]):
            script.extract()    # rip it out

        # get text
        maintext = soup.get_text()

        # break into lines and remove leading and trailing space on each
        lines = (line.strip() for line in maintext.splitlines())
        # break multi-headlines into a line each
        chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
        # drop blank lines
        maintext = '\n'.join(chunk for chunk in chunks if chunk)

        idpos = maintext.find("ID:")
        
        if(idpos == -1):
            return("Not found")
        
        
    charpos = 4
    acctoid = ""
    
    while(ord(maintext[idpos+charpos]) >= 48 and ord(maintext[idpos+charpos]) <= 57):
        acctoid += maintext[idpos+charpos]
        charpos += 1
        

    return acctoid

In [15]:
urltorefseq("https://www.ncbi.nlm.nih.gov/genome/?term=PRJNA62551")

'744'

In [16]:
import os

ncbiacc = []

acctxt = open("ncbiacc.txt")

line = acctxt.readline()

while(line):
    ncbiacc.append(line[:-1])
    line = acctxt.readline()
    
acctxt.close()

In [22]:
fullconversion = []

In [23]:


for i in range(len(ncbiacc)):
    curid = urltorefseq(ncbiacc[i])
    
    fullconversion.append([ncbiacc[i],curid])

In [30]:
def idtoname(ncbiid):
    
    fullurl = "https://www.ncbi.nlm.nih.gov/genome/plasmids/" + ncbiid + "?"
    html = urllib.request.urlopen(fullurl).read()
    soup = BeautifulSoup(html)

    # kill all script and style elements
    for script in soup(["script", "style"]):
        script.extract()    # rip it out

    # get text
    maintext = soup.get_text()

    # break into lines and remove leading and trailing space on each
    lines = (line.strip() for line in maintext.splitlines())
    # break multi-headlines into a line each
    chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
    # drop blank lines
    maintext = '\n'.join(chunk for chunk in chunks if chunk)
    
    rhizopos = maintext.find("Rhizobium")
    endpos = maintext.find("Download table")
    
    if(rhizopos == -1):
        return("Not found")
    
    if(rhizopos >= endpos):
        return("UH OH")
    
    
    name = ""
    while(rhizopos != endpos):
        name += maintext[rhizopos]
        rhizopos += 1
        
    return(name)

In [38]:
fc2 = np.copy(fullconversion)
fc2 = np.ndarray.tolist(fc2)

In [40]:
for i in range(len(fullconversion)):
    curname = idtoname(fullconversion[i][1])
    
    fc2[i].append(curname)

In [41]:
for i in range(len(fc2)):
    print(fc2[i])

['PRJNA330443', '827', 'Rhizobium etli']
['PRJNA265328', '265328', 'Not found']
['PRJNA247571', '45762', 'Rhizobium lusitanum']
['PRJNA520044', '13837', 'Rhizobium']
['PRJNA303276', '303276', 'Not found']
['PRJNA165299', '744', 'Rhizobium leguminosarum']
['PRJNA63387', '827', 'Rhizobium etli']
['PRJNA89687', '744', 'Rhizobium leguminosarum']
['PRJNA297956', '40970', 'Not found']
['PRJNA344576', '51767', 'Rhizobium oryziradicis']
['PRJNA346245', '346245', 'Not found']
['PRJNA293118', '56824', 'Rhizobium esperanzae']
['PRJNA519393', '13837', 'Rhizobium']
['PRJEB4920', '45735', 'Rhizobium favelukesii']
['PRJNA293118', '56824', 'Rhizobium esperanzae']
['PRJNA297956', '40970', 'Not found']
['PRJNA403813', '66908', 'Rhizobium hidalgonense']
['PRJNA297942', '40981', 'Not found']
['PRJNA169805', '13837', 'Rhizobium']
['PRJNA247570', '11072', 'Rhizobium tropici']
['PRJNA246118', '52711', 'Rhizobium marinum']
['PRJNA251791', '32534', 'Rhizobium vignae']
['PRJNA293118', '56824', 'Rhizobium espera

In [45]:
import os

writeme = open("fc2.txt","w")

for i in range(len(fc2)):
    writeme.write(fc2[i][0] + "\t" + fc2[i][1] + "\t" + fc2[i][2] + "\n")
    
writeme.close()
    

In [61]:
namelist = []

for i in range(len(fc2)):
    curname = fc2[i][2]
    if(curname in namelist or curname == "Not found"):
        continue
    else:
        namelist.append(curname)

In [23]:
namelist = ['Rhizobium etli',
 'Rhizobium lusitanum',
 'Rhizobium',
 'Rhizobium leguminosarum',
 'Rhizobium oryziradicis',
 'Rhizobium esperanzae',
 'Rhizobium favelukesii',
 'Rhizobium hidalgonense',
 'Rhizobium tropici',
 'Rhizobium marinum',
 'Rhizobium vignae',
 'Rhizobium freirei',
 'Rhizobium laguerreae',
 'Rhizobium undicola',
 'Rhizobium giardinii',
 'Rhizobium selenitireducens',
 'Rhizobium acidisoli',
 'Rhizobium nepotum',
 'Rhizobium grahamii',
 'Rhizobium gallicum',
 'Rhizobium mesoamericanum',
 'Rhizobium alamii',
 'Rhizobium leucaenae',
 'Rhizobium ecuadorense',
 'Rhizobium phaseoli',
 'Rhizobium altiplani',
 'Rhizobium arenae',
 'Rhizobium rhizosphaerae',
 'Rhizobium mongolense',
 'Rhizobium sullae']

In [66]:
def downloadtxt(name):
    start = "https://www.ncbi.nlm.nih.gov/genomes/Genome2BE/genome2srv.cgi?action=download&orgn=%22"
    end = "%22[orgn]&status=50|40|30|20&report=plasmids&king=All&group=All&subgroup=All&format=undefined"
    
    urlname = ""
    for i in range(len(name)):
        if((ord(name[i]) >= 65 and ord(name[i]) <= 90) or (ord(name[i]) >= 97 and ord(name[i]) <= 122)):
            urlname += name[i]
        elif(ord(name[i]) == 32):
            urlname += "%20" 
    
    fullname = start + urlname + end
    
    webbrowser.open_new(fullname)

In [71]:
downloadtxt('Rhizobium etli')

In [72]:
import time

for i in range(len(namelist)):
    downloadtxt(namelist[i])
    time.sleep(10)

In [52]:
import os
def cattext(rootdir,outtxt,specieslist):
    
    outfile = open(outtxt,'w')
    
    for subdir, dirs, files in os.walk(rootdir):
        filecount = 0
        for i in range(len(files)):
            tempO = open(os.path.join(subdir, files[i]))
            line = tempO.readline()
            listpos = str(files[i])
            if(len(listpos) == 25):
                listpos = int(listpos[-7:-5])
            elif(len(listpos) == 24):
                listpos = int(listpos[-6:-5])
            else:
                listpos = 0
            if(filecount == 0):
                if(line[-1:] == "\n"):
                    line = line[:-1]
                outfile.write(line)
                outfile.write("\t")
                outfile.write("Species plasmid page association")
                outfile.write("\n")
            line = tempO.readline()
            while(line):
                if(line[-1:] == "\n"):
                    line = line[:-1]
                outfile.write(line)
                outfile.write("\t")
                outfile.write(specieslist[listpos])
                outfile.write("\n")
                line = tempO.readline()
            filecount += 1
                    
                

In [53]:
cattext("genomes_plasmids/","fullplasmidstable.txt",namelist)

In [59]:
import os

def txttolist(txt):
    outlist = []
    text = open(txt)
    line = text.readline()
    while(line):
        curlist = []
        curstr = ""
        for i in range(len(line)):
            if((line[i] == "\n" or line[i] == "\t") and curstr):
                curlist.append(curstr)
                curstr = ""
            else:
                curstr += line[i]
        outlist.append(curlist)
        line = text.readline()
    text.close()
    return(outlist)
        
    

In [60]:
FPTlist = txttolist("fullplasmidstable.txt")

In [61]:
len(FPTlist)

452

In [64]:
print(FPTlist[0])
print(len(FPTlist[0]))

['#Organism/Name', 'Kingdom', 'Group', 'SubGroup', 'Plasmid Name', 'RefSeq', 'INSDC', 'Size (Kb)', 'GC%', 'Protein', 'rRNA', 'tRNA', 'Other RNA', 'Gene', 'Pseudogene', 'Release Date', 'Modify Date', 'Species plasmid page association']
18


In [71]:
import csv

txt_file = r"fullplasmidstable.txt"
csv_file = r"fullplasmidinf.csv"

with open(txt_file, "r") as in_text:
    in_reader = csv.reader(in_text, delimiter = '\t')
    with open(csv_file, "w") as out_csv:
        out_writer = csv.writer(out_csv)
        for row in in_reader:
            out_writer.writerow(row)

In [80]:
import os
ARS = open("fullrefseq.txt",'w')

checklist = []

count = 0
dupcount = 0
norefseq = 0

for i in range(1,len(FPTlist)):
    if(FPTlist[i][5] != "-"):
        if(FPTlist[i][5] not in checklist):
            checklist.append(FPTlist[i][5])
            ARS.write(FPTlist[i][5])
            ARS.write("\n")
            count += 1
        else:
            dupcount += 1
    else:
        norefseq += 1
ARS.close()

print("Total unique refseq count is ", count)
print("Total extra from duplications is ", dupcount)
print("Total plasmids missing refseq IDs is ", norefseq)

Total unique refseq count is  265
Total extra from duplications is  164
Total plasmids missing refseq IDs is  22


Total unique refseq count is  265
Total extra from duplications is  164
Total plasmids missing refseq IDs is  22

101 extra plasmids are found in just the "Rhizobium" download when compared against the rest


In [97]:
fullunique = []
dupcheck = []

fullunique.append(FPTlist[0])

for i in range(1,len(FPTlist)):
    if(FPTlist[i][6] not in dupcheck):
        fullunique.append(FPTlist[i])
        dupcheck.append(FPTlist[i][6])

    

In [101]:
count0 = 0

for i in range(len(fullunique)):
    if(fullunique[i][5] == '-'):
        count0 += 1

count0
    

11

In [103]:
len(fullunique) - 12

265

In [104]:
fullunique[0]

['#Organism/Name',
 'Kingdom',
 'Group',
 'SubGroup',
 'Plasmid Name',
 'RefSeq',
 'INSDC',
 'Size (Kb)',
 'GC%',
 'Protein',
 'rRNA',
 'tRNA',
 'Other RNA',
 'Gene',
 'Pseudogene',
 'Release Date',
 'Modify Date',
 'Species plasmid page association']

In [110]:
totalsize = 0.0
totalcount = 0.0

for i in range(1,len(fullunique)):
    totalsize += float(fullunique[i][7])
    totalcount += 1.0
    
totalsize/totalcount

555.763050724638

In [117]:
(555.763050724638 * 265 * 1000)/(1000 * 1000)

147.2772084420291

In [141]:
from subprocess import call
call(["ls", "-l"])

0

In [144]:

import os

test = "efetch -db nucleotide -id NZ_CP020907.1 -format fasta > plasmid_fastas/out0.fa"

os.system(test)
#2607
#3575
#7218

0

In [120]:
!ls

 conversion_functions.ipynb   fullrefseq2.txt	   ncbiacc.txt
 csvs			      fullrefseq.txt	   out1.fa
 fc2.txt		      genomes_plasmids	   test1.ipynb
 fullplasmidinf2.csv	      htmlout1.txt	   test1.txt
 fullplasmidinf.csv	     'html to txt.ipynb'
 fullplasmidstable.txt	      metadata.ipynb


In [124]:
mycode = 'print ("hello world")'
exec(mycode)



hello world


In [157]:
import os
import sys

def efetchquery(querytxt,directory):
    base1 = "efetch -db nucleotide -id "
    base2 = " -format fasta > "
    
    loopcount = 1
    
    query = open(querytxt)
    line = query.readline()
    while(line):
        print(loopcount)
        if(line[-1:] == '\n'):
            line = line[:-1]
        savename = line.replace('.','_')
        fullexe = base1 + line + base2 + directory + "/" + savename + ".fa"
        os.system(fullexe)
        loopcount += 1
        line = query.readline()
        
    query.close()
    

In [158]:
efetchquery("fullrefseq.txt","plasmid_fastas")

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265


In [161]:
import os
def catfast(rootdir,outtxt):
    
    outfile = open(outtxt,'w')
    
    for subdir, dirs, files in os.walk(rootdir):
        for i in range(len(files)):
            tempO = open(os.path.join(subdir, files[i]))
            line = tempO.readline()
            while(line):
                outfile.write(line)
                line = tempO.readline()
            outfile.write('\n')
                

In [162]:
catfast("plasmid_fastas","plasmidquery.txt")

In [163]:
fullunique

[['#Organism/Name',
  'Kingdom',
  'Group',
  'SubGroup',
  'Plasmid Name',
  'RefSeq',
  'INSDC',
  'Size (Kb)',
  'GC%',
  'Protein',
  'rRNA',
  'tRNA',
  'Other RNA',
  'Gene',
  'Pseudogene',
  'Release Date',
  'Modify Date',
  'Species plasmid page association'],
 ['Rhizobium etli',
  'Bacteria',
  'Proteobacteria',
  'Alphaproteobacteria',
  'pRetNXC12a',
  'NZ_CP020907.1',
  'CP020907',
  '182.413',
  '61.7626',
  '165',
  '-',
  '-',
  '-',
  '172',
  '7',
  '2017/05/22',
  '2017/05/22',
  'Rhizobium etli'],
 ['Rhizobium etli',
  'Bacteria',
  'Proteobacteria',
  'Alphaproteobacteria',
  'pRetNXC12b',
  'NZ_CP020908.1',
  'CP020908',
  '250.162',
  '61.4981',
  '228',
  '-',
  '-',
  '-',
  '243',
  '15',
  '2017/05/22',
  '2017/05/22',
  'Rhizobium etli'],
 ['Rhizobium etli',
  'Bacteria',
  'Proteobacteria',
  'Alphaproteobacteria',
  'pRetNXC12c',
  'NZ_CP020909.1',
  'CP020909',
  '505.136',
  '61.7238',
  '470',
  '-',
  '-',
  '-',
  '485',
  '15',
  '2017/05/22',
  '20

In [166]:
import os 

outtxt = open("fulluniquetext.txt","w")

for i in range(len(fullunique)):
    for j in range(len(fullunique[i])):
        outtxt.write(fullunique[i][j])
        if(j != (len(fullunique[i]) - 1)):
            outtxt.write("\t")
    outtxt.write("\n")
        
outtxt.close()
            
    


In [167]:
import csv

txt_file = r"fulluniquetext.txt"
csv_file = r"fulluniquetable.csv"

with open(txt_file, "r") as in_text:
    in_reader = csv.reader(in_text, delimiter = '\t')
    with open(csv_file, "w") as out_csv:
        out_writer = csv.writer(out_csv)
        for row in in_reader:
            out_writer.writerow(row)

In [168]:
countlist = []
namelist = []

for i in range(1,len(fullunique)):
    curname = fullunique[i][0]
    found = True
    for j in range(len(namelist)):
        if(curname == namelist[j]):
            countlist[j] += 1
            found = False
            continue
    if(found):
        countlist.append(1)
        namelist.append(curname)
        

In [173]:


maxlen = 0
for i in range(len(namelist)):
    if(len(namelist[i]) > maxlen):
        maxlen = len(namelist[i])

for i in range(len(namelist)):
    curspace = ""
    for j in range(maxlen - len(namelist[i])):
        curspace += " "
    print(namelist[i], curspace, "\t", countlist[i])

Rhizobium etli                                	 5
Rhizobium etli 8C-3                           	 3
Rhizobium etli bv. mimosae str. Mim1          	 6
Rhizobium etli bv. phaseoli str. IE4803       	 4
Rhizobium etli CFN 42                         	 6
Rhizobium etli CIAT 652                       	 3
Rhizobium freirei PRF 81                      	 2
Rhizobium phaseoli                            	 55
Rhizobium phaseoli Brasil 5                   	 4
Rhizobium phaseoli Ch24-10                    	 1
Rhizobium tropici CIAT 899                    	 3
Rhizobium leguminosarum                       	 24
Rhizobium leguminosarum bv. trifolii CB782    	 3
Rhizobium leguminosarum bv. trifolii WSM1325  	 5
Rhizobium leguminosarum bv. trifolii WSM1689  	 5
Rhizobium leguminosarum bv. trifolii WSM2304  	 4
Rhizobium leguminosarum bv. viciae            	 17
Rhizobium leguminosarum bv. viciae 3841       	 6
Rhizobium gallicum                            	 4
Rhizobium gallicum bv. gallicum R602          	

In [ ]:
Rhizobium etli                                	 5
Rhizobium etli 8C-3                           	 3
Rhizobium etli bv. mimosae str. Mim1          	 6
Rhizobium etli bv. phaseoli str. IE4803       	 4
Rhizobium etli CFN 42                         	 6
Rhizobium etli CIAT 652                       	 3
Rhizobium freirei PRF 81                      	 2
Rhizobium phaseoli                            	 55
Rhizobium phaseoli Brasil 5                   	 4
Rhizobium phaseoli Ch24-10                    	 1
Rhizobium tropici CIAT 899                    	 3
Rhizobium leguminosarum                       	 24
Rhizobium leguminosarum bv. trifolii CB782    	 3
Rhizobium leguminosarum bv. trifolii WSM1325  	 5
Rhizobium leguminosarum bv. trifolii WSM1689  	 5
Rhizobium leguminosarum bv. trifolii WSM2304  	 4
Rhizobium leguminosarum bv. viciae            	 17
Rhizobium leguminosarum bv. viciae 3841       	 6
Rhizobium gallicum                            	 4
Rhizobium gallicum bv. gallicum R602          	 3
Rhizobium esperanzae                          	 4
Rhizobium acidisoli                           	 4
Rhizobium favelukesii                         	 4
Rhizobium jaguaris                            	 3
Rhizobium pusense                             	 1
Rhizobium sp. 11515TR                         	 2
Rhizobium sp. ACO-34A                         	 4
Rhizobium sp. CCGE531                         	 4
Rhizobium sp. CCGE532                         	 4
Rhizobium sp. CIAT894                         	 5
Rhizobium sp. IE4771                          	 5
Rhizobium sp. IRBG74                          	 1
Rhizobium sp. Kim5                            	 4
Rhizobium sp. N113                            	 4
Rhizobium sp. N1314                           	 5
Rhizobium sp. N1341                           	 5
Rhizobium sp. N324                            	 5
Rhizobium sp. N4311                           	 5
Rhizobium sp. N541                            	 4
Rhizobium sp. N621                            	 4
Rhizobium sp. N6212                           	 4
Rhizobium sp. N731                            	 5
Rhizobium sp. N741                            	 5
Rhizobium sp. N871                            	 4
Rhizobium sp. N941                            	 4
Rhizobium sp. NT-26                           	 2
Rhizobium sp. NXC14                           	 3
Rhizobium sp. NXC24                           	 3
Rhizobium sp. TAL182                          	 5
Rhizobium sp. Y9                              	 1


In [176]:
import os 

countout = open("speciescounts.txt","w")

for i in range(len(namelist)):
    countout.write(namelist[i])
    countout.write("\t")
    countout.write(str(countlist[i]))
    countout.write("\n")
    
countout.close()

In [181]:
import os 

inf = open("TEMP.txt")
ret = open("OUT.txt",'w')

line = inf.readline()
while(line):
    curpos = 0
    while(line[curpos] != "."):
        print(curpos)
        ret.write(line[curpos])
        curpos += 1
    ret.write('\n')
    line = inf.readline()

0
1
2
3
4
5
6
7
8
9
10
0
1
2
3
4
5
6
7
8
9
10
0
1
2
3
4
5
6
7
8
9
10
0
1
2
3
4
5
6
7
8
9
10
0
1
2
3
4
5
6
7
8
9
10
0
1
2
3
4
5
6
7
8
9
10
0
1
2
3
4
5
6
7
8
9
10
0
1
2
3
4
5
6
7
8
9
10
0
1
2
3
4
5
6
7
8
0
1
2
3
4
5
6
7
8
0
1
2
3
4
5
6
7
8
0
1
2
3
4
5
6
7
8
0
1
2
3
4
5
6
7
8
0
1
2
3
4
5
6
7
8
0
1
2
3
4
5
6
7
8
9
10
0
1
2
3
4
5
6
7
8
9
10
0
1
2
3
4
5
6
7
8
9
10
0
1
2
3
4
5
6
7
8
9
10
0
1
2
3
4
5
6
7
8
0
1
2
3
4
5
6
7
8
0
1
2
3
4
5
6
7
8
0
1
2
3
4
5
6
7
8
0
1
2
3
4
5
6
7
8
0
1
2
3
4
5
6
7
8
0
1
2
3
4
5
6
7
8
0
1
2
3
4
5
6
7
8
0
1
2
3
4
5
6
7
8
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
0
1
2
3
4
5
6
7
8
9
10
0
1
2
3
4
5
6
7
8
9
10
0
1
2
3
4
5
6
7
8
9
10
0
1
2
3
4
5
6
7
8
9
10
0
1
2
3
4
5
6
7
8
9
10
0
1
2
3
4
5
6
7
8
9
10
0
1
2
3
4
5
6
7
8
9
10
0
1
2
3
4
5
6
7
8
9
10
0
1
2
3
4
5
6
7
8
9
10
0
1
2
3
4
5
6
7
8
9
10
0
1
2
3
4
5
6
7
8
9
10
0
1
2
3
4
5
6
7
8
9
10
0
1
2
3
4
5
6
7
8
9
10
0
1
2
3
4
5
6
7
8
9
10
0
1
2
3
4
5
6
7
8
9
10
0
1
2
3
4
5
6
7
8
9
10
0
1
2
3
4
5
6
7


IndexError: string index out of range

In [182]:
fullunique

[['#Organism/Name',
  'Kingdom',
  'Group',
  'SubGroup',
  'Plasmid Name',
  'RefSeq',
  'INSDC',
  'Size (Kb)',
  'GC%',
  'Protein',
  'rRNA',
  'tRNA',
  'Other RNA',
  'Gene',
  'Pseudogene',
  'Release Date',
  'Modify Date',
  'Species plasmid page association'],
 ['Rhizobium etli',
  'Bacteria',
  'Proteobacteria',
  'Alphaproteobacteria',
  'pRetNXC12a',
  'NZ_CP020907.1',
  'CP020907',
  '182.413',
  '61.7626',
  '165',
  '-',
  '-',
  '-',
  '172',
  '7',
  '2017/05/22',
  '2017/05/22',
  'Rhizobium etli'],
 ['Rhizobium etli',
  'Bacteria',
  'Proteobacteria',
  'Alphaproteobacteria',
  'pRetNXC12b',
  'NZ_CP020908.1',
  'CP020908',
  '250.162',
  '61.4981',
  '228',
  '-',
  '-',
  '-',
  '243',
  '15',
  '2017/05/22',
  '2017/05/22',
  'Rhizobium etli'],
 ['Rhizobium etli',
  'Bacteria',
  'Proteobacteria',
  'Alphaproteobacteria',
  'pRetNXC12c',
  'NZ_CP020909.1',
  'CP020909',
  '505.136',
  '61.7238',
  '470',
  '-',
  '-',
  '-',
  '485',
  '15',
  '2017/05/22',
  '20

In [183]:
#table concat

In [185]:
len(fullunique[0])

18

In [199]:
import os

nucinf = []

nuccore = open("csvs/nuccore_result.txt")
line = nuccore.readline()
line = nuccore.readline()

while(line):
    curlist = []
    
    curstrain = ""
    curword = ""
    curind = line.find("Rhi")
    while(curword != "plasmid"):
        if(line[curind] == ' ' and curword):
            curstrain += curword + ' '
            curword = ""
        else:
            curword += line[curind]
        curind += 1
    curstrain = curstrain[:-1]
    if(curstrain == "Rhizobium leguminosarum bv. viciae"):
        curstrain += " strain 3841"
    curlist.append(curstrain)
    
    line = nuccore.readline()
    line = nuccore.readline()
    
    currs = ""
    curgi = ""
    curind = 0 
    while(line[curind] != ' '):
        currs += line[curind]
        curind += 1
    curind += 4
    while(line[curind] != '\n'):
        curgi += line[curind]
        curind += 1
    curlist.append(currs)
    curlist.append(curgi)
    
    nucinf.append(curlist)
    
    line = nuccore.readline()
    line = nuccore.readline()
    

In [337]:
#CSV TO LIST

import os

oritable = []

original = open("csvs/originalgenomeinf.csv")

line = original.readline()
while(line):
    if(line[-1:] == '\n'):
        line = line[:-1]
    curline = []
    curcell = ""
    for i in range(len(line)):
        if(line[i] == ',' and line[(i-1)] != ' ' and line[(i+1)] != ' '):
            curline.append(curcell)
            curcell = ""
        else:
            curcell += line[i]
    curline.append(curcell)
    oritable.append(curline)
    line = original.readline()

In [217]:
for i in range(len(nucinf)):
    print(nucinf[i][0])

Rhizobium etli CFN 42
Rhizobium etli CFN 42
Rhizobium etli CFN 42
Rhizobium etli CFN 42
Rhizobium etli CFN 42
Rhizobium etli CFN 42
Rhizobium leguminosarum bv. viciae strain 3841
Rhizobium leguminosarum bv. viciae strain 3841
Rhizobium leguminosarum bv. viciae strain 3841
Rhizobium leguminosarum bv. viciae strain 3841
Rhizobium leguminosarum bv. viciae strain 3841
Rhizobium leguminosarum bv. viciae strain 3841
Rhizobium etli CIAT 652
Rhizobium etli CIAT 652
Rhizobium etli CIAT 652
Rhizobium leguminosarum bv. trifolii WSM2304
Rhizobium leguminosarum bv. trifolii WSM2304
Rhizobium leguminosarum bv. trifolii WSM2304
Rhizobium leguminosarum bv. trifolii WSM2304
Rhizobium leguminosarum bv. trifolii WSM1325
Rhizobium leguminosarum bv. trifolii WSM1325
Rhizobium leguminosarum bv. trifolii WSM1325
Rhizobium leguminosarum bv. trifolii WSM1325
Rhizobium leguminosarum bv. trifolii WSM1325
Rhizobium tropici CIAT 899
Rhizobium tropici CIAT 899
Rhizobium tropici CIAT 899
Rhizobium freirei PRF 81
Rhi

In [230]:
redlist = []
for i in range(len(nucinf)):
    if([nucinf[i][0]] not in redlist):
        redlist.append([nucinf[i][0]])

In [348]:
redlist = [['Rhizobium etli CFN 42', 0],
 ['Rhizobium leguminosarum bv. viciae strain 3841', 0],
 ['Rhizobium etli CIAT 652', 0],
 ['Rhizobium leguminosarum bv. trifolii WSM2304', 0],
 ['Rhizobium leguminosarum bv. trifolii WSM1325', 0],
 ['Rhizobium tropici CIAT 899', 114],
 ['Rhizobium freirei PRF 81', 29],
 ['Rhizobium etli bv. mimosae str. Mim1', 259],
 ['Rhizobium sp. IRBG74', 143],
 ['Rhizobium etli bv. phaseoli str. IE4803', 44],
 ['Rhizobium sp. IE4771', 52],
 ['Rhizobium sp. LPU83', 16],
 ['Rhizobium leguminosarum bv. trifolii WSM1689', 160],
 ['Rhizobium leguminosarum bv. trifolii CB782', 244],
 ['Rhizobium phaseoli Ch24-10', 180],
 ['Rhizobium sp. NT-26', 48],
 ['Rhizobium sp. N731', 25],
 ['Rhizobium sp. N621', 212],
 ['Rhizobium sp. N6212', 207],
 ['Rhizobium sp. N1314', 58],
 ['Rhizobium sp. N1341', 128],
 ['Rhizobium sp. N113', 155],
 ['Rhizobium phaseoli strain N261', 264],
 ['Rhizobium phaseoli strain N831', 14],
 ['Rhizobium esperanzae strain N561', 72],
 ['Rhizobium phaseoli strain R620', 17],
 ['Rhizobium phaseoli strain R630', 80],
 ['Rhizobium phaseoli strain R650', 123],
 ['Rhizobium phaseoli strain R723', 213],
 ['Rhizobium phaseoli strain N841', 172],
 ['Rhizobium phaseoli strain N671', 113],
 ['Rhizobium phaseoli strain R611', 221],
 ['Rhizobium phaseoli strain N161', 61],
 ['Rhizobium sp. N324', 238],
 ['Rhizobium phaseoli strain R744', 139],
 ['Rhizobium phaseoli strain N931', 193],
 ['Rhizobium sp. N871', 235],
 ['Rhizobium phaseoli strain N771', 256],
 ['Rhizobium sp. N741', 69],
 ['Rhizobium sp. N941', 215],
 ['Rhizobium sp. N541', 183],
 ['Rhizobium leguminosarum strain Vaf-108', 0],
 ['Rhizobium etli 8C-3', 140],
 ['Rhizobium gallicum strain IE4872', 122],
 ['Rhizobium sp. CIAT894', 121],
 ['Rhizobium phaseoli Brasil 5', 0],
 ['Rhizobium sp. NXC14', 120],
 ['Rhizobium sp. TAL182', 0],
 ['Rhizobium etli strain NXC12', 0],
 ['Rhizobium sp. Kim5', 104],
 ['Rhizobium leguminosarum bv. viciae strain BIHB 1148', 0],
 ['Rhizobium leguminosarum bv. viciae strain BIHB 1217', 0],
 ['Rhizobium sp. N4311', 0],
 ['Rhizobium sp. 11515TR', 101],
 ['Rhizobium sp. ACO-34A', 102],
 ['Rhizobium sp. Y9', 31],
 ['Rhizobium sp. NXC24', 144],
 ['Rhizobium leguminosarum bv. viciae strain UPM791', 0],
 ['Rhizobium leguminosarum strain Norway', 0],
 ['Rhizobium leguminosarum strain ATCC 14479', 0],
 ['Rhizobium sp. CCGE531', 0],
 ['Rhizobium jaguaris strain CCGE525', 0],
 ['Rhizobium sp. CCGE532', 0],
 ['Rhizobium acidisoli strain FH23', 79],
 ['Rhizobium pusense strain CFBP5875', 0]]

In [232]:
tempcur = 0

In [316]:
tempcount = 0
for i in range(len(redlist)):
    if(redlist[i][1] != 0):
        tempcount += 1
print(tempcount)
print(len(redlist))
print(len(redlist)-tempcount)


46
65
19


In [338]:
import csv
with open('csvs/taxontable.csv','r') as f:
    reader = csv.reader(f)
    taxtab = list(reader)

In [350]:
for i in range(len(redlist)):
    if(redlist[i][1] != 0):
        foundcheck = True
        genome = oritable[redlist[i][1]][0]
        for j in range(1,len(taxtab)):
            if(taxtab[j][0] == genome):
                foundcheck = False
                redlist[i][1] = j
        if(foundcheck):
            print(redlist[i])

In [370]:
redlist = [['Rhizobium etli CFN 42', 283],
 ['Rhizobium leguminosarum bv. viciae strain 3841', 123],
 ['Rhizobium etli CIAT 652', 150],
 ['Rhizobium leguminosarum bv. trifolii WSM2304', 29],
 ['Rhizobium leguminosarum bv. trifolii WSM1325', 3],
 ['Rhizobium tropici CIAT 899', 118],
 ['Rhizobium freirei PRF 81', 30],
 ['Rhizobium etli bv. mimosae str. Mim1', 280],
 ['Rhizobium sp. IRBG74', 153],
 ['Rhizobium etli bv. phaseoli str. IE4803', 45],
 ['Rhizobium sp. IE4771', 54],
 ['Rhizobium sp. LPU83', 16],
 ['Rhizobium leguminosarum bv. trifolii WSM1689', 172],
 ['Rhizobium leguminosarum bv. trifolii CB782', 265],
 ['Rhizobium phaseoli Ch24-10', 193],
 ['Rhizobium sp. NT-26', 50],
 ['Rhizobium sp. N731', 26],
 ['Rhizobium sp. N621', 230],
 ['Rhizobium sp. N6212', 224],
 ['Rhizobium sp. N1314', 60],
 ['Rhizobium sp. N1341', 135],
 ['Rhizobium sp. N113', 166],
 ['Rhizobium phaseoli strain N261', 286],
 ['Rhizobium phaseoli strain N831', 15],
 ['Rhizobium esperanzae strain N561', 73],
 ['Rhizobium phaseoli strain R620', 17],
 ['Rhizobium phaseoli strain R630', 82],
 ['Rhizobium phaseoli strain R650', 129],
 ['Rhizobium phaseoli strain R723', 231],
 ['Rhizobium phaseoli strain N841', 184],
 ['Rhizobium phaseoli strain N671', 117],
 ['Rhizobium phaseoli strain R611', 239],
 ['Rhizobium phaseoli strain N161', 63],
 ['Rhizobium sp. N324', 257],
 ['Rhizobium phaseoli strain R744', 147],
 ['Rhizobium phaseoli strain N931', 208],
 ['Rhizobium sp. N871', 254],
 ['Rhizobium phaseoli strain N771', 277],
 ['Rhizobium sp. N741', 70],
 ['Rhizobium sp. N941', 233],
 ['Rhizobium sp. N541', 197],
 ['Rhizobium leguminosarum strain Vaf-108', 0],
 ['Rhizobium etli 8C-3', 148],
 ['Rhizobium gallicum strain IE4872', 128],
 ['Rhizobium sp. CIAT894', 126],
 ['Rhizobium phaseoli Brasil 5', 210],
 ['Rhizobium sp. NXC14', 125],
 ['Rhizobium sp. TAL182', 0],
 ['Rhizobium etli strain NXC12', 0],
 ['Rhizobium sp. Kim5', 108],
 ['Rhizobium leguminosarum bv. viciae strain BIHB 1148', 0],
 ['Rhizobium leguminosarum bv. viciae strain BIHB 1217', 0],
 ['Rhizobium sp. N4311', 0],
 ['Rhizobium sp. 11515TR', 104],
 ['Rhizobium sp. ACO-34A', 105],
 ['Rhizobium sp. Y9', 32],
 ['Rhizobium sp. NXC24', 154],
 ['Rhizobium leguminosarum bv. viciae strain UPM791', 0],
 ['Rhizobium leguminosarum strain Norway', 0],
 ['Rhizobium leguminosarum strain ATCC 14479', 0],
 ['Rhizobium sp. CCGE531', 0],
 ['Rhizobium jaguaris strain CCGE525', 0],
 ['Rhizobium sp. CCGE532', 0],
 ['Rhizobium acidisoli strain FH23', 81],
 ['Rhizobium pusense strain CFBP5875', 0]]

In [371]:
for i in range(len(redlist)):
    if(redlist[i][1] == 0):
        print(redlist[i][0] + "    " + str(i))

Rhizobium leguminosarum strain Vaf-108    41
Rhizobium sp. TAL182    47
Rhizobium etli strain NXC12    48
Rhizobium leguminosarum bv. viciae strain BIHB 1148    50
Rhizobium leguminosarum bv. viciae strain BIHB 1217    51
Rhizobium sp. N4311    52
Rhizobium leguminosarum bv. viciae strain UPM791    57
Rhizobium leguminosarum strain Norway    58
Rhizobium leguminosarum strain ATCC 14479    59
Rhizobium sp. CCGE531    60
Rhizobium jaguaris strain CCGE525    61
Rhizobium sp. CCGE532    62
Rhizobium pusense strain CFBP5875    64


In [488]:
import numpy as np
fullunique2 = np.copy(fullunique)
fullunique2 = np.ndarray.tolist(fullunique2)

In [377]:
for i in range(len(redlist)):
    for j in range(len(nucinf)):
        if(nucinf[j][0] == redlist[i][0]):            ###
            nucinf[j].append(redlist[i][1])

In [457]:
print(fullunique2[0])

['#Organism/Name', 'Kingdom', 'Group', 'SubGroup', 'Plasmid Name', 'RefSeq', 'INSDC', 'Size (Kb)', 'GC%', 'Protein', 'rRNA', 'tRNA', 'Other RNA', 'Gene', 'Pseudogene', 'Release Date', 'Modify Date', 'Species plasmid page association']


In [382]:
print(taxtab[0])

['taxon_oid', 'Domain', 'Sequencing Status', 'Study Name', 'Genome Name / Sample Name', 'Sequencing Center', 'IMG Genome ID ', 'Genus', 'Species', 'IMG Submission ID', 'JGI Project ID / ITS SP ID', 'JGI Analysis Product Name', 'JGI Analysis Project Type', 'GOLD Analysis Project ID', 'GOLD Analysis Project Type', 'GOLD Sequencing Project ID', 'GOLD Study ID', 'Assembly Method', 'Genome Completeness %', 'High Quality', 'Is Public', 'NCBI Taxon ID', 'NCBI Project ID', 'RefSeq Project ID', 'Release Date', 'Strain', 'Biotic Relationships', 'Culture Type', 'Cultured', 'Ecosystem', 'Ecosystem Category', 'Ecosystem Subtype', 'Ecosystem Type', 'Ecotype', 'Geographic Location', 'GOLD Sequencing Depth', 'GOLD Sequencing Quality', 'GOLD Sequencing Strategy', 'Habitat', 'Host Gender', 'Host Name', 'Isolation', 'Isolation Country', 'Latitude', 'Longitude', 'NCBI Bioproject Accession', 'NCBI Biosample Accession', 'pH', 'Project / Study Name', 'Pubmed ID', 'Sequencing Method', 'Type Strain', 'Uncultur

In [489]:
fullunique2[0][(len(fullunique2[0])-1)] = "tempind"
for i in range(1,len(fullunique2)):
    for j in range(len(nucinf)):
        if(fullunique2[i][5] == nucinf[j][1]):
            fullunique2[i][17] = nucinf[j][3]

In [459]:
len(fullunique2[0]) -1 

17

In [435]:
print(len(nucinf))
print(len(fullunique2))

if(type(12) == int):
    print('yuh')


265
277
yuh


In [460]:
for i in range(len(fullunique2)):
    print(fullunique2[i][17])

tempind
0
0
0
0
0
148
148
148
280
280
280
280
280
280
45
45
45
45
283
283
283
283
283
283
150
150
150
30
30
286
286
286
286
15
15
15
15
63
63
63
63
147
147
147
147
82
82
82
82
231
231
231
231
17
17
17
17
208
208
208
208
117
117
117
117
117
129
129
129
129
277
277
277
277
277
184
184
184
184
184
239
239
239
239
210
210
210
210
193
118
118
118
Rhizobium leguminosarum
Rhizobium leguminosarum
Rhizobium leguminosarum
Rhizobium leguminosarum
Rhizobium leguminosarum
Rhizobium leguminosarum
Rhizobium leguminosarum
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
265
265
265
3
3
3
3
3
172
172
172
172
172
29
29
29
29
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
123
123
123
123
123
123
128
128
128
128
Rhizobium gallicum
Rhizobium gallicum
Rhizobium gallicum
73
73
73
73
81
81
81
81
16
16
16
Rhizobium
0
0
0
0
104
104
105
105
105
105
0
0
0
0
0
0
0
0
126
126
126
126
126
54
54
54
54
54
153
108
108
108
108
166
166
166
166
60
60
60
60
60
135
135
135
135
135
257
257
257
257
257
0
0
0
0
0
197
197
197
197
230
230
230
230
224
224
22

In [445]:
print(redlist[1])
print(nucinf[1])
print(fullunique2[1])

['Rhizobium leguminosarum bv. viciae strain 3841', 123]
['Rhizobium etli CFN 42', 'NC_007763.1', '86359881', 283]
['Rhizobium etli', 'Bacteria', 'Proteobacteria', 'Alphaproteobacteria', 'pRetNXC12a', 'NZ_CP020907.1', 'CP020907', '182.413', '61.7626', '165', '-', '-', '-', '172', '7', '2017/05/22', '2017/05/22', 0]


In [490]:
found = []

for i in range(1,len(nucinf)):
    for j in range(1,len(fullunique2)):
        if(nucinf[i][1] == fullunique2[j][5]):
            fullunique2[j][0] = nucinf[i][0]
            found.append(j)

notfound = []

for i in range(1,len(fullunique2)):
    if(i not in found):
        notfound.append(i)
        
print(len(notfound))
for i in range(len(notfound)):
    print(notfound[i])

12
20
93
94
95
96
97
98
99
161
162
163
175


In [491]:
for i in range(len(fullunique2)):
    if(len(fullunique2[i]) != 18):
        print(i)

In [493]:
fullunique2[0]

['#Organism/Name',
 'Kingdom',
 'Group',
 'SubGroup',
 'Plasmid Name',
 'RefSeq',
 'INSDC',
 'Size (Kb)',
 'GC%',
 'Protein',
 'rRNA',
 'tRNA',
 'Other RNA',
 'Gene',
 'Pseudogene',
 'Release Date',
 'Modify Date',
 'tempind']

In [492]:
for i in range(len(fullunique)):
    if(fullunique[i][5] == '-'):
        print(i)

93
94
95
96
97
98
99
161
162
163
175


In [468]:
fullunique[20][5]

'NC_007762.1'

In [477]:
print(fullunique2[20][17])

283


In [499]:
for i in range(1,len(fullunique2)):
    if(type(fullunique2[i][17]) != int):
        print(i)

In [497]:
notfound

[93, 94, 95, 96, 97, 98, 99, 161, 162, 163, 175]

In [496]:
del notfound[0]

In [498]:
for i in range(len(notfound)):
    curind = len(notfound) - 1 - i
    del fullunique2[notfound[curind]]

In [500]:
len(fullunique) - len(fullunique2)

11

reordering "redlist" to correspond to order of species in "fullunique2"

this will allow for easier copmarison between redlist and fullunique2

ill be appending the information from the full rhizobium jgi info "taxtab" onto the end of redlist instead of fullunique2 - this will be easier to read. Fullunique2 has many duplications of the same '#Organism/Name' label to describe differing info for the different plasmids the taxtab info would be repeated my times

ill most likely be inserting a new column at x_ind = 0 for both fullunique2 and redlist to label each organsim/name name with a number. This will allow for even easier comparisons between the two tables 

In [507]:
for i in range(len(fullunique2)):
    print(fullunique2[i][0])

#Organism/Name
Rhizobium etli strain NXC12
Rhizobium etli strain NXC12
Rhizobium etli strain NXC12
Rhizobium etli strain NXC12
Rhizobium etli strain NXC12
Rhizobium etli 8C-3
Rhizobium etli 8C-3
Rhizobium etli 8C-3
Rhizobium etli bv. mimosae str. Mim1
Rhizobium etli bv. mimosae str. Mim1
Rhizobium etli bv. mimosae str. Mim1
Rhizobium etli bv. mimosae str. Mim1
Rhizobium etli bv. mimosae str. Mim1
Rhizobium etli bv. mimosae str. Mim1
Rhizobium etli bv. phaseoli str. IE4803
Rhizobium etli bv. phaseoli str. IE4803
Rhizobium etli bv. phaseoli str. IE4803
Rhizobium etli bv. phaseoli str. IE4803
Rhizobium etli CFN 42
Rhizobium etli CFN 42
Rhizobium etli CFN 42
Rhizobium etli CFN 42
Rhizobium etli CFN 42
Rhizobium etli CFN 42
Rhizobium etli CIAT 652
Rhizobium etli CIAT 652
Rhizobium etli CIAT 652
Rhizobium freirei PRF 81
Rhizobium freirei PRF 81
Rhizobium phaseoli strain N261
Rhizobium phaseoli strain N261
Rhizobium phaseoli strain N261
Rhizobium phaseoli strain N261
Rhizobium phaseoli strain

In [511]:
#finding order in fullunique2. I know all species are already grouped

fu2order = []

for i in range(1,len(fullunique2)):
    if(fullunique2[i][0] not in fu2order):
        fu2order.append(fullunique2[i][0])

In [534]:
#reordering redlist as redlist2

print(len(redlist))
print(len(fu2order))    #len should be equal

reorderinds = []

for i in range(len(redlist)):
    for j in range(len(fu2order)):
        if(redlist[i][0] == fu2order[j]):
            print(redlist[i][0])
            print(fu2order[j])
            reorderinds.append(j)
            continue 
import numpy as np
redlist2 = np.copy(redlist)
redlist2 = np.ndarray.tolist(redlist2)

for i in range(len(redlist2)):
    redlist2[reorderinds[i]] = redlist[i]
    


65
65
Rhizobium etli CFN 42
Rhizobium etli CFN 42
Rhizobium leguminosarum bv. viciae strain 3841
Rhizobium leguminosarum bv. viciae strain 3841
Rhizobium etli CIAT 652
Rhizobium etli CIAT 652
Rhizobium leguminosarum bv. trifolii WSM2304
Rhizobium leguminosarum bv. trifolii WSM2304
Rhizobium leguminosarum bv. trifolii WSM1325
Rhizobium leguminosarum bv. trifolii WSM1325
Rhizobium tropici CIAT 899
Rhizobium tropici CIAT 899
Rhizobium freirei PRF 81
Rhizobium freirei PRF 81
Rhizobium etli bv. mimosae str. Mim1
Rhizobium etli bv. mimosae str. Mim1
Rhizobium sp. IRBG74
Rhizobium sp. IRBG74
Rhizobium etli bv. phaseoli str. IE4803
Rhizobium etli bv. phaseoli str. IE4803
Rhizobium sp. IE4771
Rhizobium sp. IE4771
Rhizobium sp. LPU83
Rhizobium sp. LPU83
Rhizobium leguminosarum bv. trifolii WSM1689
Rhizobium leguminosarum bv. trifolii WSM1689
Rhizobium leguminosarum bv. trifolii CB782
Rhizobium leguminosarum bv. trifolii CB782
Rhizobium phaseoli Ch24-10
Rhizobium phaseoli Ch24-10
Rhizobium sp. NT

In [539]:
print(redlist2[0])
print(fu2order[0])
print(fullunique2[1])

['Rhizobium etli strain NXC12', 0]
Rhizobium etli strain NXC12
['Rhizobium etli strain NXC12', 'Bacteria', 'Proteobacteria', 'Alphaproteobacteria', 'pRetNXC12a', 'NZ_CP020907.1', 'CP020907', '182.413', '61.7626', '165', '-', '-', '-', '172', '7', '2017/05/22', '2017/05/22', 0]


In [542]:
redlist2.insert(0,['#Organism/Name','tempind'])

In [546]:
#insert number indentification

redlist2[0].insert(0,'Organism number')

for i in range(1,len(redlist2)):
    redlist2[i].insert(0,(i-1))

In [556]:
for i in range(len(taxtab[0])):
    redlist2[0].append(taxtab[0][i])
    
for i in range(1,len(redlist2)):
    if(redlist2[i][2] == 0):
        for j in range(len(taxtab[0])):
            redlist2[i].append('')
    else:
        corind = redlist2[i][2]
        for j in range(len(taxtab[corind])):
            redlist2[i].append(taxtab[corind][j])
    

In [561]:
for i in range(len(redlist2)):
    if(len(redlist2[i]) != 58):
        print('oof')

In [564]:
for i in range(len(redlist2)):
    del redlist2[i][2]

In [569]:
#add number identification to fullunique

fullunique2[0].insert(0,'Organism number')

for i in range(1,len(fullunique2)):
    for j in range(1,len(redlist2)):
        if(fullunique2[i][0] == redlist2[j][1]):
            fullunique2[i].insert(0,redlist2[j][0])
            continue

In [576]:
for i in range(len(fullunique2)):
    if(len(fullunique2[i]) != 19):
        print(i)

In [580]:
#remove 'tempind' column
for i in range(len(fullunique2)):
    del(fullunique2[i][18])

In [587]:
#save fullunique2, retlist as text and csv files

#LIST TO TEXT

import os

pitxt = open("csvs/plasmidinf.txt",'w')

for i in range(len(fullunique2)):
    for j in range(len(fullunique2[i])):
        pitxt.write(str(fullunique2[i][j]))
        if(j < (len(fullunique2[i]) - 1)):
            pitxt.write('\t')
    pitxt.write('\n')
    
    
oitxt = open("csvs/organisminf.txt",'w')

for i in range(len(redlist2)):
    for j in range(len(redlist2[i])):
        oitxt.write(str(redlist2[i][j]))
        if(j < (len(redlist2[i]) - 1)):
            oitxt.write('\t')
    oitxt.write('\n')
    

In [589]:
import csv

#TEXT TO CSV

txt_file = r"csvs/organisminf.txt"
csv_file = r"csvs/organisminf.csv"

with open(txt_file, "r") as in_text:
    in_reader = csv.reader(in_text, delimiter = '\t')
    with open(csv_file, "w") as out_csv:
        out_writer = csv.writer(out_csv)
        for row in in_reader:
            out_writer.writerow(row)
            

txt_file = r"csvs/plasmidinf.txt"
csv_file = r"csvs/plasmidinf.csv"

with open(txt_file, "r") as in_text:
    in_reader = csv.reader(in_text, delimiter = '\t')
    with open(csv_file, "w") as out_csv:
        out_writer = csv.writer(out_csv)
        for row in in_reader:
            out_writer.writerow(row)

In [588]:
len(fullunique2)

266